In [1]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

## Loading Datasets of twiiter and Bitcoin prices

In [2]:
Twdf1=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/tweetsfinal1.csv',error_bad_lines=False,engine = 'python',header = None) 
Twdf2=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/tweetsfinal2.csv',error_bad_lines=False,engine = 'python',header = None) 
Twdf3=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/tweetsfinal3.csv',error_bad_lines=False,engine = 'python',header = None) 
Twdf2 = Twdf2.drop([2,3], axis=1)
Twdf3 = Twdf3.drop([2,3], axis=1)
print(Twdf1.head())
print(Twdf3.head())
print(Twdf2.head())
TwDF = pd.concat([Twdf1,Twdf2,Twdf3])

Skipping line 845142: unexpected end of data
Skipping line 352574: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 353631: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead
Skipping line 883146: unexpected end of data


                              0  \
0  Thu Nov 09 17:43:41 EST 2017   
1  Thu Nov 09 17:43:40 EST 2017   
2  Thu Nov 09 17:43:39 EST 2017   
3  Thu Nov 09 17:43:39 EST 2017   
4  Thu Nov 09 17:43:39 EST 2017   

                                                   1  
0  RT @Forbes: The Failure of SegWit2x Shows Bitc...  
1  RT @mindstatex: Lots of love from unknown mine...  
2  RT @FernandoHuamanX: Warning: Built-in Keylogg...  
3  RT @LevelNetwork: Join our telegram. All infor...  
4  RT @realsheepwolf: $DIGAF: FLOAT=16M, THE "ONL...  
                              0  \
0  Mon Nov 27 17:40:38 EST 2017   
1  Mon Nov 27 17:40:38 EST 2017   
2  Mon Nov 27 17:40:38 EST 2017   
3  Mon Nov 27 17:40:37 EST 2017   
4  Mon Nov 27 17:40:36 EST 2017   

                                                   1  
0  RT @daneelproject: Another day, another high s...  
1  RT @BankExProtocol: Be prepared to BKX #tokens...  
2  (Bitcoin Price Analysis: Bitcoin Tests Parabol...  
3  RT @daneelproject: Daneel

In [3]:
# FullDataTw = pd.merge(Twdf1,Twdf2, on=0, how='outer')#,Twdf3,
#FullDataTw.shape

In [4]:
BtcDF=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/BitCoinPrice.csv',error_bad_lines=False,engine = 'python',header = None) 
FullDataTw=sql.createDataFrame(TwDF)
FullDataBtc=sql.createDataFrame(BtcDF) #creating pandas df and then changing it to pyspark df

In [5]:
FullDataTw = FullDataTw.dropna() #getting rid of full empty rows
#print(FullDataTw.count())
#print(FullDataBtc.count())

In [6]:
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
FullDataTw = FullDataTw.withColumnRenamed('0', 'DateTime') #setting column names of Twitter dataset
FullDataTw = FullDataTw.withColumnRenamed('1', 'Tweet')
FullDataBtc = FullDataBtc.withColumnRenamed('0', 'DateTime') #setting column names of Bitcoin price dataset
FullDataBtc = FullDataBtc.withColumnRenamed('1', 'Price')
FullDataBtc = FullDataBtc.filter(FullDataBtc.DateTime != 'Date') #to get rid of first row with the header

## Pre-Processing Twitter dataframe

In [7]:
Tw_samp = FullDataTw #taking sample of 50 rows and working on it

In [8]:
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
def function_udf(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['Tweet']))
CleanDF.show(5)

+--------------------+--------------------+--------------------+
|            DateTime|               Tweet|       CleanedTweets|
+--------------------+--------------------+--------------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [9]:
from textblob import TextBlob  #passing cleaned tweets and getting a sentiment score for each tweet
def senti_score_udf(input_str):
    analysis = TextBlob(input_str)
    return analysis.sentiment.polarity
func_udf2 = udf(senti_score_udf, FloatType())
CleanDF = CleanDF.withColumn('Sentiment_score', func_udf2(CleanDF['CleanedTweets']))
CleanDF.show(5)

+--------------------+--------------------+--------------------+---------------+
|            DateTime|               Tweet|       CleanedTweets|Sentiment_score|
+--------------------+--------------------+--------------------+---------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|    -0.18888889|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|     0.25833333|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|            0.0|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|            0.0|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|          -0.05|
+--------------------+--------------------+--------------------+---------------+
only showing top 5 rows



In [10]:
def Tw_Time_format(stri):  #manipulating and casting the strings(DateTime) of tweets dataframe to timestamps
    dic = {'Nov':'11','Oct':'10'}
    ans = ''
    ans += stri[-4:]+'-'+ dic[stri[4:7]]+'-'+stri[8:19]
    return ans
func_udf3 = udf(Tw_Time_format,StringType())
CleanDF = CleanDF.withColumn('DateTime_c', func_udf3(CleanDF['DateTime']))
CleanDF = CleanDF.withColumn("DateTime_casted",CleanDF['DateTime_c'].cast(TimestampType()))
CleanDF.show(5)

+--------------------+--------------------+--------------------+---------------+-------------------+-------------------+
|            DateTime|               Tweet|       CleanedTweets|Sentiment_score|         DateTime_c|    DateTime_casted|
+--------------------+--------------------+--------------------+---------------+-------------------+-------------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|    -0.18888889|2017-11-09 17:43:41|2017-11-09 17:43:41|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|     0.25833333|2017-11-09 17:43:40|2017-11-09 17:43:40|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|            0.0|2017-11-09 17:43:39|2017-11-09 17:43:39|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|            0.0|2017-11-09 17:43:39|2017-11-09 17:43:39|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|          -0.05|2017-11-09 17:43:39|2017-11-09 17:43:39|
+--------------------+----------

In [11]:
FinalTw = CleanDF.selectExpr("DateTime_casted as Date_Time", "CleanedTweets as Cleaned_Tweets","Sentiment_score")
FinalTw.show(5) #selecting necessary columns

+-------------------+--------------------+---------------+
|          Date_Time|      Cleaned_Tweets|Sentiment_score|
+-------------------+--------------------+---------------+
|2017-11-09 17:43:41|The Failure of Se...|    -0.18888889|
|2017-11-09 17:43:40|Lots of love from...|     0.25833333|
|2017-11-09 17:43:39|Warning Built in ...|            0.0|
|2017-11-09 17:43:39|Join our telegram...|            0.0|
|2017-11-09 17:43:39|DIGAF FLOAT 16M T...|          -0.05|
+-------------------+--------------------+---------------+
only showing top 5 rows



## Pre-Processing Bitcoin dataframe

In [12]:
from datetime import datetime 
from dateutil import parser
def Btc_Time_format(input_str): #manipulating and casting the strings(DateTime) of BTC dataframe to timestamps
    input_str = re.sub(r'/17','', input_str)
    input_str = '2017-'+ input_str
    input_str = re.sub(r'/', '-', input_str)
    input_str += ':00'
    return input_str[:10]+""+input_str[10:]
func_udf = udf(Btc_Time_format, StringType())
FullDataBtc = FullDataBtc.withColumn('Cleaned_BTC_Time', func_udf(FullDataBtc['DateTime']))
FullDataBtc.show(5)

+-------------+-------+------------------+
|     DateTime|  Price|  Cleaned_BTC_Time|
+-------------+-------+------------------+
|10/30/17 0:00|6123.21|2017-10-30 0:00:00|
|10/30/17 1:00|6131.35|2017-10-30 1:00:00|
|10/30/17 2:00|6114.17|2017-10-30 2:00:00|
|10/30/17 3:00|6153.11|2017-10-30 3:00:00|
|10/30/17 4:00|6151.09|2017-10-30 4:00:00|
+-------------+-------+------------------+
only showing top 5 rows



In [13]:
CleandfBtc = FullDataBtc.withColumn("Cleaned_BTC_Time_New",FullDataBtc['Cleaned_BTC_Time'].cast(TimestampType()))
FinalBtc = CleandfBtc.selectExpr("Cleaned_BTC_Time_New as Date_Time", "Price")
FinalBtc = FinalBtc.withColumn("Price",FinalBtc['Price'].cast(DoubleType()))
FinalBtc.show(5)#In this cell, casting to timesstamp, changing col names and casting price type to double

+-------------------+-------+
|          Date_Time|  Price|
+-------------------+-------+
|2017-10-30 00:00:00|6123.21|
|2017-10-30 01:00:00|6131.35|
|2017-10-30 02:00:00|6114.17|
|2017-10-30 03:00:00|6153.11|
|2017-10-30 04:00:00|6151.09|
+-------------------+-------+
only showing top 5 rows



## Dataframes Look like this...

In [14]:
FinalTw.printSchema()

root
 |-- Date_Time: timestamp (nullable = true)
 |-- Cleaned_Tweets: string (nullable = true)
 |-- Sentiment_score: float (nullable = true)



In [15]:
FinalBtc.printSchema()
FinalBtc.count()

root
 |-- Date_Time: timestamp (nullable = true)
 |-- Price: double (nullable = true)



672

## Truncating timestamps to hours and then grouping them by hour

In [16]:
dt_truncated = ((round(unix_timestamp(col('Date_Time')) / 3600) * 3600).cast('timestamp'))
FinalTw = FinalTw.withColumn('dt_truncated', dt_truncated)
FinalTw = FinalTw.selectExpr("dt_truncated as Date_Time","Cleaned_Tweets","Sentiment_score")
UTC = ((unix_timestamp(col('Date_Time'))+ 5*60*60).cast('timestamp'))
FinalTw = FinalTw.withColumn('UTC', UTC)
FinalTw = FinalTw.selectExpr("UTC as Date_Time","Cleaned_Tweets","Sentiment_score")
FinalTw.show(10)

+-------------------+--------------------+---------------+
|          Date_Time|      Cleaned_Tweets|Sentiment_score|
+-------------------+--------------------+---------------+
|2017-11-09 23:00:00|The Failure of Se...|    -0.18888889|
|2017-11-09 23:00:00|Lots of love from...|     0.25833333|
|2017-11-09 23:00:00|Warning Built in ...|            0.0|
|2017-11-09 23:00:00|Join our telegram...|            0.0|
|2017-11-09 23:00:00|DIGAF FLOAT 16M T...|          -0.05|
|2017-11-09 23:00:00|My luggage likes ...|            0.0|
|2017-11-09 23:00:00|As Bitcoin become...|           0.55|
|2017-11-09 23:00:00|A crucial feature...|            0.1|
|2017-11-09 23:00:00|As Bitcoin become...|           0.55|
|2017-11-09 23:00:00|As Bitcoin become...|           0.55|
+-------------------+--------------------+---------------+
only showing top 10 rows



In [18]:
#FinalTw.registerTempTable("temp")
#FinalTw_avg = sql.sql("SELECT Date_Time As DateTime, AVG(Sentiment_score) As Sentiment_score FROM temp GROUP BY Date_Time")
FinalTw_avg = FinalTw.select("Date_Time","Sentiment_score").groupBy("Date_Time").agg(avg(col("Sentiment_score")))
FinalTw_avg.show(5)

KeyboardInterrupt: 

In [ ]:
#This cell is just to collect all the corpus of an hour. Just did this for the future work.
from pyspark.sql import functions as f
df_with_text = FinalTw.groupby("Date_Time").agg(f.concat_ws(" ", f.collect_list(FinalTw.Cleaned_Tweets)))
df_with_text.show(5)

In [ ]:
# FinalTw_avg.count()
# from pyspark.sql.functions import *
# df_sort = FinalTw_avg.sort(asc("Date_Time"))
# df_sort.show()

## Joining twitter and bitcoin dataframes by DateTime

In [ ]:
FinalTw_avg.registerTempTable("avgs")
FinalBtc.registerTempTable("prices")
results = sql.sql("SELECT DateTime,Sentiment_score,Price FROM avgs JOIN prices ON avgs.DateTime = prices.Date_Time order by avgs.DateTime")
results.show(5)

In [ ]:
results.count()

In [ ]:
results.repartition(1).write.csv("One.csv")#this will write df to single csv instead of writing diff csv acc to partitions 

In [ ]:
FinalTw_avg = FinalTw.select("Date_Time","Sentiment_score").groupBy("Date_Time").agg(avg(col("Sentiment_score")))
FinalTw_avg = FinalTw_avg.selectExpr("Date_Time as date", "`avg(Sentiment_score)` as score")